In [1]:
import numpy as np
import pandas as pd
import pickle
from flask import Flask, request, jsonify
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import os

In [2]:
# Step 1: Load dataset
# We are using the Iris dataset for simplicity. In real scenarios, you will load a dataset from a file.
data = load_iris()
X = data.data  # Features
y = data.target  # Target variable

In [5]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [6]:
# Step 2: Split dataset into training and testing sets
# 80% training, 20% testing to evaluate model performance.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Step 3: Define models and parameter grids
# We define 3 different models with their respective hyperparameters for tuning.
models = {
    "RandomForest": (Pipeline([
        ('clf', RandomForestClassifier())
    ]), {
        'clf__n_estimators': [50, 100, 200],
        'clf__max_depth': [None, 10, 20]
    }),
    "SVM": (Pipeline([
        ('scaler', StandardScaler()),  # StandardScaler is used to normalize features for SVM
        ('clf', SVC())
    ]), {
        'clf__C': [0.1, 1, 10],
        'clf__kernel': ['linear', 'rbf']
    }),
    "GradientBoosting": (Pipeline([
        ('clf', GradientBoostingClassifier())
    ]), {
        'clf__n_estimators': [50, 100, 200],
        'clf__learning_rate': [0.01, 0.1, 0.2]
    })
}

best_models = {}

In [8]:
# Step 4: Perform Grid Search to find the best parameters
# This step will take some time as it runs cross-validation on multiple hyperparameters.
for model_name, (pipeline, param_grid) in models.items():
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")


Best parameters for RandomForest: {'clf__max_depth': 10, 'clf__n_estimators': 200}
Best parameters for SVM: {'clf__C': 0.1, 'clf__kernel': 'linear'}
Best parameters for GradientBoosting: {'clf__learning_rate': 0.01, 'clf__n_estimators': 50}


In [9]:
# Step 5: Evaluate all models to find the best performing one
best_model = None
best_score = 0

for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"Accuracy of {model_name}: {acc:.4f}")
    if acc > best_score:
        best_score = acc
        best_model = (model_name, model)

print(f"\nBest Model: {best_model[0]} with accuracy {best_score:.4f}")

Accuracy of RandomForest: 1.0000
Accuracy of SVM: 1.0000
Accuracy of GradientBoosting: 1.0000

Best Model: RandomForest with accuracy 1.0000


In [10]:
# Step 6: Save the best model to a pickle file
# This pickle file is used for model inference in Flask API.
with open('best_model.pkl', 'wb') as file:
    pickle.dump(best_model[1], file)